# Classification of Leukaemia by Gene Expression

## Import Libraries

In [66]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re
import seaborn as sns

## Import Data

In [67]:
# Training Dataset
training_data = pd.read_csv('Data/data_set_ALL_AML_train.csv')
training_data.head()

,Gene Description,Gene Accession Number,1,call,2,call.1,3,call.2,4,call.3,...,29,call.33,30,call.34,31,call.35,32,call.36,33,call.37
0,AFFX-BioB-5_at (endogenous control),AFFX-BioB-5_at,-214,A,-139,A,-76,A,-135,A,...,15,A,-318,A,-32,A,-124,A,-135,A
1,AFFX-BioB-M_at (endogenous control),AFFX-BioB-M_at,-153,A,-73,A,-49,A,-114,A,...,-114,A,-192,A,-49,A,-79,A,-186,A
2,AFFX-BioB-3_at (endogenous control),AFFX-BioB-3_at,-58,A,-1,A,-307,A,265,A,...,2,A,-95,A,49,A,-37,A,-70,A
3,AFFX-BioC-5_at (endogenous control),AFFX-BioC-5_at,88,A,283,A,309,A,12,A,...,193,A,312,A,230,P,330,A,337,A
4,AFFX-BioC-3_at (endogenous control),AFFX-BioC-3_at,-295,A,-264,A,-376,A,-419,A,...,-51,A,-139,A,-367,A,-188,A,-407,A


In [ ]:
# Labels for the Training Set
labels = pd.read_csv('Data/actual.csv', index_col='patient')
labels.head()

: 

In [ ]:
# Test Dataset
test_data = pd.read_csv('Data/data_set_ALL_AML_independent.csv')
test_data.head()

: 

## Exploration of Data

### Re-Formatting of Data

In [ ]:
# Training Dataset

: 

In [ ]:
# Test Dataset


: 

- Gene Description: This column contains a description of the gene
- Gene Accession Number: This column contains the unique identifier for each gene
- pt: This columns contains the patient number
- expr:
- call:
- call_val:This column is an assessment of the presence or absence of gene expression based on the signal intensity for each gene in each sample - "Present" (P), "Absent" (A), and "Marginal" (M)
- patient: This columns also contains the patient number
- cancer: This column indicates the type of cancer for each patient

### Distribution of Call Values

The distribution for the Absent (A) genes are heavily skewed.
### Distribution of Classes

In [ ]:
classes_count = labels.groupby('cancer')['patient'].nunique()
classes_count

: 

The classes are very imbalanced.